In [1]:
import pandas as pd 
import folium
import os
import geohash
from folium.plugins import HeatMap
from sklearn import datasets, linear_model
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, mean_squared_log_error
from datetime import datetime
import numpy as np

In [2]:
RESULTS_DIR = "results"

In [3]:
df = pd.read_csv(os.path.join(RESULTS_DIR, "10_million_with_elevation_geohashes_timestamps.csv"))

In [4]:
df.head()

,Unnamed: 0,value,geohash,geohash5,timestamp
0,0,23.0,xn7tncn498k2,xn7tn,1.540076e+09
1,1,19.0,xnezj2x7rnh7,xnezj,1.540076e+09
2,2,15.0,xneysek46d7w,xneys,1.540076e+09
3,3,16.0,u336qgr9tvzw,u336q,1.540076e+09
4,4,18.0,xneyyr50hk99,xneyy,1.540076e+09


In [5]:
df = df[df.value >= 0]
df.head()

,Unnamed: 0,value,geohash,geohash5,timestamp
0,0,23.0,xn7tncn498k2,xn7tn,1.540076e+09
1,1,19.0,xnezj2x7rnh7,xnezj,1.540076e+09
2,2,15.0,xneysek46d7w,xneys,1.540076e+09
3,3,16.0,u336qgr9tvzw,u336q,1.540076e+09
4,4,18.0,xneyyr50hk99,xneyy,1.540076e+09


In [23]:
points_geohashes_prec_5 = df.geohash5.value_counts()
points_geohashes_prec_5.mean(), points_geohashes_prec_5.median()

(310.47663253261874, 34.0)

In [29]:
geohashes_for_regression = list(points_geohashes_prec_5.where(lambda count: count <= 1000).dropna().where(lambda count: count >= 50).dropna().index)
len(geohashes_for_regression)

10834

In [30]:
groupped_df = df[df.geohash5.isin(geohashes_for_regression)][["value", "geohash5", "timestamp"]].groupby("geohash5")
len(groupped_df)

10834

In [88]:
def prepare_svr_regresion_model(data):
    data = data.sort_values(by=['timestamp'])
    new_data = pd.DataFrame(data=[[]])
    size = data.shape[0]
    new_data['count'] = size
    
    if size <= 1:
        new_data['regresion'] = None
        new_data['reg_mean_squared_error'] = None
        new_data['reg_r2_score_error'] = None
        new_data['reg_median_absolute_error'] = None
        new_data['reg_mean_squared_log_error'] = None
        return new_data
    
    train_size = int(0.8 * size)
    test_size = size - train_size
    
    X_train = np.array(data.timestamp[:train_size]).reshape(-1, 1)
    X_test = np.array(data.timestamp[train_size:]).reshape(-1, 1)
    
    Y_train = np.ravel(np.array(data.value[:train_size]).reshape(-1, 1))
    Y_test = np.ravel(np.array(data.value[train_size:]).reshape(-1, 1))
    
    regr = SVR(kernel='linear')

    regr.fit(X_train, Y_train)
    new_data['regresion'] = regr
    
    Y_pred = regr.predict(X_test)
    new_data['reg_mean_squared_error'] = count_function_of_none(mean_squared_error, Y_test, Y_pred)
    new_data['reg_r2_score_error'] = count_function_of_none(r2_score, Y_test, Y_pred)
    new_data['reg_median_absolute_error'] = count_function_of_none(median_absolute_error, Y_test, Y_pred)
    new_data['reg_mean_squared_log_error'] = count_function_of_none(mean_squared_log_error, Y_test, Y_pred)
    
    return new_data

def count_function_of_none(fun, Y_test, Y_pred):
    try:
        return fun(Y_test, Y_pred)
    except ValueError:
        return np.finfo(np.float64).max

In [89]:
groupped_df2 = groupped_df.head(n=len(groupped_df)).head(n=len(groupped_df))
groupped_df2.head()

,value,geohash5,timestamp
19,21.0,u0tyv,1.540076e+09
2479,40.0,u0tyv,1.540069e+09
5137,31.0,u0tyv,1.540062e+09
6451,44.0,u0tyv,1.540058e+09
7741,25.0,u0tyv,1.540055e+09


In [90]:
len(groupped_df2)

10834

In [91]:
groupped_df2 = groupped_df2.groupby("geohash5")

In [92]:
len(groupped_df2)

135

In [ ]:
df_after_training = groupped_df.apply(prepare_svr_regresion_model).reset_index()

In [ ]:
df_after_training.head()

In [ ]:
df_after_training_mean_squared = df_after_training[df_after_training.reg_mean_squared_error <= 100]
df_after_training_reg_r2_score = df_after_training[df_after_training.reg_r2_score_error <= 100][df_after_training.reg_r2_score_error >= -10]
df_after_training_median_absolute = df_after_training[df_after_training.reg_median_absolute_error <= 10]
df_after_training_mean_log = df_after_training[df_after_training.reg_mean_squared_log_error <= 10]

In [ ]:
def get_coordinates(geo):
    bbox = geohash.bbox(geo)
    return [[bbox["w"], bbox["n"]], [bbox["e"], bbox["n"]], [bbox["e"], bbox["s"]], [bbox["w"], bbox["s"]]]

def get_geo_data(ghash_df):
    features = []

    for geo, *_ in ghash_df.values:
        features.append({"type": "Feature", "id": geo, 
                         "geometry": {"type": "Polygon", "coordinates": [get_coordinates(geo)]}})

    return {"type": "FeatureCollection", "features": features}

def visualize_with_folium(df, error):
    m = folium.Map(location=[37.760806, 140.474722], zoom_start=10)

    m.choropleth(
        geo_data=get_geo_data(df),
        name='choropleth',
        data=df,
        columns=['geohash5', error],
        key_on='feature.id',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='error'
    )
    folium.LayerControl().add_to(m)
    return m

In [ ]:
m_mean_squared = visualize_with_folium(df_after_training_mean_squared, 'reg_mean_squared_error')
m_reg_r2_score = visualize_with_folium(df_after_training_reg_r2_score, 'reg_r2_score_error')
m_median_absolute = visualize_with_folium(df_after_training_median_absolute, 'reg_median_absolute_error')
m_mean_log = visualize_with_folium(df_after_training_mean_log, 'reg_median_absolute_error')

In [ ]:
m_mean_squared.save(os.path.join(RESULTS_DIR, "rbf_poly_regression_mean_squared_errors.html"))
m_reg_r2_score.save(os.path.join(RESULTS_DIR, "rbf_poly_regression_reg_r2_score_errors.html"))
m_median_absolute.save(os.path.join(RESULTS_DIR, "rbf_poly_regression_median_absolute_errors.html"))
m_mean_log.save(os.path.join(RESULTS_DIR, "rbf_poly_regression_mean_log_errors.html"))